In [1]:
import os
os.chdir('..')

In [2]:
data_folder = '../data/'
server_folder = './server/'
template_folder = './templates/'

import numpy as np
import json

# For the Python notebook
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [3]:
directions = ['grad', 'hess', 'bfgs', 'bfgs-inv', 'hybrid', 'hybrid-inv']
types = ['LS', 'LS-ABS', 'TR', 'TR-ABS']

unfeasible = ['TR_grad', 'TR-ABS_grad', 'LS_hybrid', 'TR_hybrid', 'TR_bfgs-inv', 'TR-ABS_bfgs-inv', 'LS_hybrid-inv', 'TR_hybrid-inv', 'TR-ABS_hybrid-inv']

combinations = []

# Add all combinations
for d in directions:
    for t in types:
        str_ = t + '_' + d
        
        if str_ not in unfeasible:
            combinations.append(str_)

In [4]:
models = ['MNL', 
          'Nested',
          'LPMC_DrivingCost_small', 
          'LPMC_DrivingCost', 
          'LPMC_DrivingCost_big',
          'LPMC_RemoveRest_small',
          'LPMC_RemoveRest',
          'LPMC_RemoveRest_big',
          'LPMC_Full_small',
          'LPMC_Full',
          'LPMC_Full_big']

# Best against Biogeme

In [22]:
results = {}

for mod in models:    
    results[mod] = {}
    
    times = {'avg': 99999, 'std': 999, 'name': None}
    epochs = {'avg': 99999, 'std': 999, 'name': None}

    try:
        with open(server_folder + mod + '/results/biogeme_bfgs.json', 'r') as infile:
            data = json.load(infile)

        avg_time = np.mean(data['time'])
        avg_epochs = np.mean(data['epochs'])

        times['bio_avg'] = np.mean(data['time'])
        times['bio_std'] = np.std(data['time'])
        epochs['bio_avg'] = np.mean(data['epochs'])
        epochs['bio_std'] = np.std(data['epochs'])
        
    except FileNotFoundError:
        
        times['bio_avg'] = 100
        times['bio_std'] = 9

        epochs['bio_avg'] = 100
        epochs['bio_std'] = 9

    
    for comb in combinations:
                
        try:
            with open(server_folder + mod + '/results/' + comb + '.json', 'r') as infile:
                data = json.load(infile)

            avg_time = np.mean(data['time'])
            
            if avg_time < times['avg']:
                times['avg'] = avg_time
                times['std'] = np.std(data['time'])
                times['name'] = comb
                epochs['avg'] = np.mean(data['epochs'])
                epochs['std'] = np.std(data['epochs'])
                epochs['name'] = comb
                
            
            """
            avg_epochs = np.mean(data['epochs'])
            
            if avg_epochs < epochs['avg']:
                epochs['avg'] = avg_epochs
                epochs['std'] = np.std(data['epochs'])
                epochs['name'] = comb
            """

        
        except FileNotFoundError:
            pass
        
    if times['avg'] > times['bio_avg']:
        times['ratio'] = "$\\times {:.2f}$".format(times['avg']/times['bio_avg'])
    else:
        times['ratio'] = "$\div {:.2f}$".format(times['bio_avg']/times['avg'])
        
    if epochs['avg'] > epochs['bio_avg']:
        epochs['ratio'] = "$\\times {:.2f}$".format(epochs['avg']/epochs['bio_avg'])
    else:
        epochs['ratio'] = "$\div {:.2f}$".format(epochs['bio_avg']/epochs['avg'])
    
    results[mod]['time'] = times
    results[mod]['epochs'] = epochs

In [23]:
for s in ['time', 'epochs']:
    print("Results for {}:".format(s))
        
    for mod in models:
        res = results[mod][s]
        
        print('  ' + mod + ':')
        print('    Best algorithm: {}'.format(res['name']))
        
        if 'LPMC_RemoveRest' in mod or 'LPMC_Full' in mod or mod == 'MTMC':
            print('    Value: ${:.2f} \pm {:.2f}$'.format(res['avg'], res['std']))
            print('    Biogeme: ${:.2f} \pm {:.2f}$'.format(res['bio_avg'], res['bio_std']))
        else:  
            print('    Value: ${:.3f} \pm {:.3f}$'.format(res['avg'], res['std']))
            print('    Biogeme: ${:.3f} \pm {:.3f}$'.format(res['bio_avg'], res['bio_std']))
            
        print('    Improvement: {}'.format(res['ratio']))
    
    print('\n')

Results for time:
  MNL:
    Best algorithm: TR_hess
    Value: $0.113 \pm 0.005$
    Biogeme: $0.067 \pm 0.012$
    Improvement: $\times 1.70$
  Nested:
    Best algorithm: TR_hess
    Value: $0.636 \pm 0.012$
    Biogeme: $0.517 \pm 0.071$
    Improvement: $\times 1.23$
  LPMC_DrivingCost_small:
    Best algorithm: TR_hess
    Value: $0.471 \pm 0.005$
    Biogeme: $1.656 \pm 0.135$
    Improvement: $\div 3.52$
  LPMC_DrivingCost:
    Best algorithm: TR_hess
    Value: $0.883 \pm 0.012$
    Biogeme: $2.794 \pm 0.133$
    Improvement: $\div 3.16$
  LPMC_DrivingCost_big:
    Best algorithm: TR_hess
    Value: $1.276 \pm 0.010$
    Biogeme: $4.320 \pm 0.168$
    Improvement: $\div 3.39$
  LPMC_RemoveRest_small:
    Best algorithm: LS-ABS_hybrid-inv
    Value: $13.85 \pm 1.01$
    Biogeme: $62.11 \pm 0.50$
    Improvement: $\div 4.48$
  LPMC_RemoveRest:
    Best algorithm: LS-ABS_hybrid-inv
    Value: $23.73 \pm 1.23$
    Biogeme: $120.09 \pm 0.65$
    Improvement: $\div 5.06$
  LPMC_Remo

# Tables in the appendix

In [24]:
for mod in models:
    print(mod + ':')
    for comb in combinations:
        
        print('  ' + comb + ':')

        
        try:
            with open(server_folder + mod + '/results/' + comb + '.json', 'r') as infile:
                data = json.load(infile)

            avg_time = np.mean(data['time'])
            std_time = np.std(data['time'])

            if 'LPMC' in mod:
                print('    time: ${:.2f} \pm {:.2f}$ [s]'.format(avg_time, std_time))
            else:  
                print('    time: ${:.3f} \pm {:.3f}$ [s]'.format(avg_time, std_time))

            avg_epochs = np.mean(data['epochs'])
            std_epochs = np.std(data['epochs'])
            
            if 'ABS' in comb:
                print('    epochs: ${:.2f} \pm {:.2f}$'.format(avg_epochs, std_epochs))
            else:
                print('    epochs: ${:d}$'.format(int(avg_epochs)))
        
        except FileNotFoundError:
            print('\x1b[0;30;41m' + '    File not found...' + '\x1b[0m')
    print('\n')

MNL:
  LS_grad:
    time: $0.908 \pm 0.018$ [s]
    epochs: $39$
  LS-ABS_grad:
    time: $1.014 \pm 0.153$ [s]
    epochs: $30.85 \pm 6.66$
  LS_hess:
    time: $0.240 \pm 0.007$ [s]
    epochs: $8$
  LS-ABS_hess:
    time: $0.464 \pm 0.031$ [s]
    epochs: $11.25 \pm 0.78$
  TR_hess:
    time: $0.113 \pm 0.005$ [s]
    epochs: $5$
  TR-ABS_hess:
    time: $0.300 \pm 0.019$ [s]
    epochs: $7.25 \pm 0.28$
  LS_bfgs:
    time: $0.474 \pm 0.008$ [s]
    epochs: $12$
  LS-ABS_bfgs:
    time: $0.623 \pm 0.049$ [s]
    epochs: $12.56 \pm 1.30$
  TR_bfgs:
    time: $0.710 \pm 0.007$ [s]
    epochs: $31$
  TR-ABS_bfgs:
    time: $0.782 \pm 0.073$ [s]
    epochs: $27.04 \pm 3.10$
  LS_bfgs-inv:
    time: $0.416 \pm 0.014$ [s]
    epochs: $12$
  LS-ABS_bfgs-inv:
    time: $0.510 \pm 0.026$ [s]
    epochs: $10.63 \pm 0.87$
  LS-ABS_hybrid:
    time: $0.592 \pm 0.046$ [s]
    epochs: $12.68 \pm 1.39$
  TR-ABS_hybrid:
    time: $0.596 \pm 0.051$ [s]
    epochs: $19.44 \pm 2.14$
  LS-ABS_hybrid-in